In [ ]:
import json

from bs4 import BeautifulSoup
from requests import get

page: int = 1

cities: dict = {}

while True:
    query: str = f'https://www.posta.hr/pretrazivanje-postanskih-ureda/263?pojam=&page={page}'

    postal_codes_page = get(query)

    soup = BeautifulSoup(postal_codes_page.content, 'html.parser')

    table = soup.find("table", attrs={"class": "tablica-borders"})

    # the address has an "infinite" number of pages
    # the last page's table only has 1 tr, with table heading, no data
    if len(table.findAll('tr')) <= 1:
        break

    trs: list = table.find_all("tr")

    trs = trs[1:]

    for tag in trs:

        text: str = tag.text

        postal_code: str = text[:5].strip()

        comma: int = text.find(',')

        city: str = text[5:comma].strip()
        # if city == '':
        #    continue

        print(postal_code + ' ' + city)

        if city not in cities.keys():
            cities[city] = [postal_code]
        else:
            cities[city].append(postal_code)

    print(str(page) + '---------------------' + str(len(trs)))
    page += 1
    trs = []


# codes_file = open('pcodes.json', encoding='UTF-8', mode='w')

# json.dump(cities, codes_file)

# codes_file.close()

In [10]:
import json

codes_file = open('pcodes.json', encoding='UTF-8', mode='r')
cities_file = open('cities_fixtures.json', encoding='UTF-8', mode='r')

json_codes = json.load(codes_file)
json_cities = json.load(cities_file)

for i, city in enumerate(json_cities):
    cname: str = city['fields']['city_name']

    pcode: str = json_codes.get(cname, '00000')

    city['fields']['postal_code'] = pcode[0]

    json_cities[i] = city

# batch of unmatched cities
bad: list = [city for city in json_cities if city['fields']['postal_code'] == '0']


for i, city in enumerate(bad):

    for city2, code in json_codes.items():
        bracket: int = city2.find('(')

        # some cities have an alias name (eg. cities with italian names)
        # so remove the alias part by only taking the city name up to the index of the bracket
        if bracket != -1:
            cname: str = city2[:bracket-1]
        else:
            cname: str = city2

        # some cities in json_cities start with 'GRAD' ('CITY'), but not in json_codes
        # so add 'GRAD ' to city name search term
        if city['fields']['city_name'].startswith('GRAD'):
            cname = 'GRAD ' + cname

        # some places are prefixed with 'ZAGREB-'
        # remove prefix
        if cname.startswith('ZAGREB-'):
            cname = cname[len('ZAGREB-'):]

        # one place has a name with a different dialect across json_cities and json_codes
        # equalize dialects
        if cname.find('NEDELJA') != -1:
            cname = cname.replace('NEDELJA', 'NEDJELJA')

        # only set postal code for city which doesn't already have one
        if cname == city['fields']['city_name'] and city['fields']['postal_code'] == '0':
            bad[i]['fields']['postal_code'] = code
        else:
            continue

TypeError: string indices must be integers

In [38]:
import json

with open('isplate.json', mode='r', encoding='UTF-8') as d:
    t_rates = json.load(d)
    d.close()

with open('pcodes.json', mode='r', encoding='UTF-8') as p:
    p_codes = json.load(p)
    p.close()

p_codes = {city.encode('UTF-8').upper(): codes for city, codes in p_codes.items()}

for joppd, city in t_rates.items():

    for city2, code in p_codes.items():

        bracket: int = city2.find('(')

        # some cities have an alias name (eg. cities with italian names)
        # so remove the alias part by only taking the city name up to the index of the bracket
        if bracket != -1:
            cname: str = city2[:bracket-1]
        else:
            cname: str = city2

        # some cities in json_cities start with 'GRAD' ('CITY'), but not in json_codes
        # so add 'GRAD ' to city name search term
        if city['mjesto'].startswith('GRAD'):
            cname = 'GRAD ' + cname

        # some places are prefixed with 'ZAGREB-'
        # remove prefix
        if cname.startswith('ZAGREB-'):
            cname = cname[len('ZAGREB-'):]

        # one place has a name with a different dialect across json_cities and json_codes
        # equalize dialects
        if cname.find('NEDELJA') != -1:
            cname = cname.replace('NEDELJA', 'NEDJELJA')

        # only set postal code for city which doesn't already have one
        if cname == city['mjesto']:
            t_rates[joppd]['postal_code'] = code[0]

{"b'ZAGREB'": ['10000', '10101', '10103', '10104', '10105', '10106', '10108', '10109', '10110', '10111', '10112', '10113', '10114', '10115', '10116', '10119', '10120', '10121', '10122', '10123', '10124', '10126', '10128', '10129', '10130', '10131', '10132', '10133', '10134', '10135', '10136', '10137', '10138', '10139', '10140', '10141', '10142', '10144', '10145', '10146', '10147', '10148', '10151', '10153', '10156', '10158', '10160', '10162', '10163', '10166', '10168', '10169', '10172', '10173', '10174', '10175'], "b'ZAGREB-CARINA'": ['10004'], "b'ZAGREB-SLOBO\\xc5\\xa0TINA'": ['10010'], "b'ZAGREB-NOVI ZAGREB'": ['10020'], "b'ZAGREB-DUBRAVA'": ['10040'], "b'ZAGREB-SUSEDGRAD'": ['10090'], "b'ZAGREB-ZRA\\xc4\\x8cNA LUKA'": ['10150'], "b'LU\\xc4\\x8cKO'": ['10250'], "b'HRVATSKI LESKOVAC'": ['10251'], "b'KUPINE\\xc4\\x8cKI KRALJEVEC'": ['10252'], "b'DONJI DRAGONO\\xc5\\xbdEC'": ['10253'], "b'ODRA'": ['10254'], "b'GORNJI STUPNIK'": ['10255'], "b'BOTINEC'": ['10256'], "b'BREZOVICA'": ['10257

In [18]:
print(t_rates)

{'00019': {'mjesto': 'ANDRIJA\\XC5\\XA1EVCI', 'prirez': 8.0, 'iban': 'HR1910010051700112008'}, '00027': {'mjesto': 'ANTUNOVAC', 'prirez': 0.0, 'iban': 'HR8410010051700212009', 'postal_code': '31216'}, '00035': {'mjesto': 'BABINA GREDA', 'prirez': 5.0, 'iban': 'HR3110010051700312000', 'postal_code': '32276'}, '00043': {'mjesto': 'BAKAR', 'prirez': 0.0, 'iban': 'HR4210010051700412003', 'postal_code': '51222'}, '00051': {'mjesto': 'BALE-VALLE', 'prirez': 1.0, 'iban': 'HR6410010051700512002'}, '00060': {'mjesto': 'BARBAN', 'prirez': 5.0, 'iban': 'HR4810010051700612006', 'postal_code': '52207'}, '00078': {'mjesto': 'BARILOVI\\XC4\\X87', 'prirez': 0.0, 'iban': 'HR9710010051700712004'}, '00086': {'mjesto': 'BA\\XC5\\XA1KA', 'prirez': 0.0, 'iban': 'HR6510010051700812005'}, '00094': {'mjesto': 'BA\\XC5\\XA1KA VODA', 'prirez': 0.0, 'iban': 'HR7110010051700912001'}, '00108': {'mjesto': 'BEBRINA', 'prirez': 0.0, 'iban': 'HR4410010051701012009', 'postal_code': '35254'}, '00116': {'mjesto': 'BEDEKOV

In [1]:
import json

with open('isplate.json', encoding='UTF-8', mode='r') as c:
    cities: dict = json.load(c)
    c.close()

with open('pcodes.json', encoding='UTF-8', mode='r') as c:
    postal_codes: dict = json.load(c)
    c.close()

In [2]:
for joppd, city in cities.items():

    bracket: int = city['mjesto'].find('(')

    if bracket != -1 and city['mjesto'] != 'SVETA NEDELJA (SAMOBOR)':
        cname: str = city['mjesto'][:bracket-1]
    else:
        cname: str = city['mjesto']

    cities[joppd]['mjesto'] = cname

for city, pcode in postal_codes.items():

    bracket: int = city.find('(')

    if bracket != -1:
        cname: str = city[:bracket-1]
    else:
        cname: str = city

    if 'ZAGREB' in cname and cname.find('-') != -1:
        cname = cname[:cname.find('-')-1]

    if cname in postal_codes.keys():
        continue
    else:
        postal_codes[cname] = postal_codes.pop(city)

In [3]:
for joppd, city in cities.items():

    if city['mjesto'] == 'SVETA NEDELJA (SAMOBOR)':
        cities[joppd]['postal_code'] = postal_codes['SVETA NEDELJA'][0]
        continue

    if city['mjesto'] in postal_codes.keys():
        cities[joppd]['postal_code'] = postal_codes[city['mjesto']][0]
    else:
        cities[joppd]['postal_code'] = 'NULL'

In [5]:
with open('isplate.json', encoding='UTF-8', mode='r') as c:
    cities2: dict = json.load(c)
    c.close()

for joppd, data in cities.items():
    cities[joppd]['mjesto'] = cities2[joppd]['mjesto']

In [7]:
"""
with open('cities2.json', mode='w', encoding='UTF-8') as d:
    d.write(json.dumps(cities))
    d.close()
"""

In [16]:
import json

with open('cities2.json', mode='r', encoding='UTF-8') as c:
    cities2: dict = json.load(c)
    c.close()

with open('pcodes.json', mode='r', encoding='UTF-8') as c:
    pcodes: dict = json.load(c)
    c.close()

In [17]:
# get cities with missing data
cities = {joppd: data for joppd, data in cities2.items() if '-' in data['mjesto'] and data['postal_code'] == 'NULL'}

print(len(cities))

for joppd, data in cities.items():

    dash: int = data['mjesto'].find('-')

    fixed_name: str = data['mjesto'][:dash] + ' (' + data['mjesto'][dash+1:] + ')'

    if fixed_name in pcodes.keys():
        cities2[joppd]['postal_code'] = pcodes[fixed_name][0]
        print(data)

23
{'mjesto': 'BALE-VALLE', 'prirez': 1.0, 'iban': 'HR6410010051700512002', 'postal_code': '52211'}
{'mjesto': 'BRTONIGLA-VERTENEGLIO', 'prirez': 0.0, 'iban': 'HR2910010051704012004', 'postal_code': '52474'}
{'mjesto': 'BUJE-BUIE', 'prirez': 6.0, 'iban': 'HR0310010051704212001', 'postal_code': '52460'}
{'mjesto': 'FAŽANA-FASANA', 'prirez': 3.0, 'iban': 'HR8710010051761912006', 'postal_code': '52212'}
{'mjesto': 'FUNTANA-FONTANE', 'prirez': 0.0, 'iban': 'HR6410010051762912005', 'postal_code': '52452'}
{'mjesto': 'GROŽNJAN-GRISIGNANA', 'prirez': 0.0, 'iban': 'HR2410010051713812004', 'postal_code': '52429'}
{'mjesto': 'LIŽNJAN-LISIGNANO', 'prirez': 5.0, 'iban': 'HR9410010051723512005', 'postal_code': '52204'}
{'mjesto': 'MOTOVUN-MONTONA', 'prirez': 0.0, 'iban': 'HR6610010051727412006', 'postal_code': '52424'}
{'mjesto': 'NOVIGRAD-CITTANOVA', 'prirez': 0.0, 'iban': 'HR7810010051729112006', 'postal_code': '52466'}
{'mjesto': 'OPRTALJ-PORTOLE', 'prirez': 0.0, 'iban': 'HR2410010051730412002',

In [23]:
cities = {joppd: data for joppd, data in cities2.items() if data['postal_code'] == 'NULL' and 'DUBRAVA' in data['mjesto']}

for joppd, data in cities.items():

    cities2[joppd] = data

{}


In [24]:
with open('cities2.json', mode='w', encoding='UTF-8') as d:
    data = json.dumps(cities2)

    d.write(data)
    d.close()

In [1]:
import json

with open('cities2.json', mode='r', encoding='UTF-8') as d:

    dta: dict = json.load(d)

    fixture: list = []

    for joppd, data in dta.items():
        fixture.append({
            "model": "third_parties_app.city",
            "pk": joppd,
            "fields": {
              "city_name": data['mjesto'],
              "postal_code": data['postal_code'],
              "iban": data['iban'],
              "tax_rate": data['prirez']
            }
        })

    res = json.dumps(fixture)

    r = open('city_fixtures.json', mode='w', encoding='UTF-8')
    r.write(res)
    r.close()

    d.close()
